In [1]:
!pip install tensorflow-gpu
!pip install Keras

In [2]:
!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [3]:
import tensorflow as tf
print(tf.__version__)


2022-08-23 16:06:46.779358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-23 16:06:46.779379: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.9.1


In [4]:
import torch as tc
print(tc.__version__)


1.10.2


In [5]:
from tensorflow import keras 

In [6]:
#import keras
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Lambda, Reshape
from tensorflow.keras.layers import SimpleRNN,Bidirectional,BatchNormalization
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.losses import KLDivergence,binary_crossentropy
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import glob,os
from tensorflow.keras.layers import Dense,GRU, Flatten, Conv2D, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import (Callback, TensorBoard, EarlyStopping,
                             ModelCheckpoint, CSVLogger, ProgbarLogger)
from tensorflow.keras.layers import (Input, Dense, TimeDistributed, LSTM, GRU, Dropout,  
                         Concatenate, Flatten, RepeatVector, Lambda, Bidirectional, SimpleRNN)#merge,
import sys
import csv
#from collections import Iterable, OrderedDict
import datetime
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

In [7]:
try:
    from collections.abc import Iterable
except ImportError:
    from collections import Iterable

In [8]:
print(np.__version__)


1.21.6


In [9]:
try:
    lcs_file ='lcs.npy'
except ImportError:
    print('please use npy file only.')

In [10]:

#lcs_file ='lcs.npy'
lcs =np.load(lcs_file,allow_pickle=True)
print(len(lcs))
print(lcs[10000][0:5])


18382
[[5.82022673e+04 1.92292023e+01 6.47380054e-02]
 [5.82022742e+04 1.91007309e+01 5.87802716e-02]
 [5.82072615e+04 1.92533073e+01 6.59256652e-02]
 [5.82102655e+04 1.93263912e+01 6.96673542e-02]
 [5.82102735e+04 1.91409512e+01 6.05796054e-02]]


In [11]:

#shuffling the data to avoid systematics 
np.random.seed(42)
np.random.shuffle(lcs)
lcs[100][0:5]


array([[5.82022764e+04, 1.85910397e+01, 4.75157984e-02],
       [5.82052526e+04, 1.86000938e+01, 4.78192680e-02],
       [5.82052715e+04, 1.85581970e+01, 4.64340188e-02],
       [5.82062694e+04, 1.85672989e+01, 4.67308350e-02],
       [5.82082824e+04, 1.84853821e+01, 4.41396125e-02]])

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
lcs_raw = pad_sequences(lcs, value=np.nan, dtype='float', padding='post')
#print(lcs_raw[100][0:5])
#print(lcs_raw[:,:,1])
#print(np.nanmax(lcs_raw[:, :, 1], axis=1)) this is magnitude value
#print(np.inf)
#print(np.all(np.isnan(lcs_raw[:, :, 1])) | (np.nanmax(lcs_raw[:, :, 1], axis=1) > np.inf))

In [13]:
#functions taken from https://github.com/yutarotachibana/CatalinaQSO_AutoEncoder

def times_to_lags(T):
    """(N x n_step) matrix of times -> (N x n_step) matrix of lags.
    First time is assumed to be zero.
    """
    assert T.ndim == 2, "T must be an (N x n_step) matrix"
    return np.c_[np.diff(T, axis=1)/365., np.zeros(T.shape[0])]


In [14]:
def preprocess(X_raw, m_max=np.inf):
    X = X_raw.copy()
    #print(X)
    wrong_units =  np.all(np.isnan(X[:, :, 1])) | (np.nanmax(X[:, :, 1], axis=1) > m_max)
    #print(wrong_units)
    X = X[~wrong_units, :, :]
    X[:, :, 0] = times_to_lags(X[:, :, 0])
    #print(X[:, :, 0])
    means = np.atleast_2d(np.nanmean(X[:, :, 1], axis=1)).T
    #print(means)
    X[:, :, 1] -= means
    scales = np.atleast_2d(np.nanstd(X[:, :, 1], axis=1)).T
    #print(scales)
    X[:, :, 1] /= scales
    errors = X[:, :, 2] / scales
    #print(errors)
    X = X[:, :, :2]
    #print(X) is lcs_scaled value
    return X, means, scales, errors, wrong_units

In [15]:
lcs_scaled, means, scales, errors, wrong_units = preprocess(lcs_raw)

In [16]:
print(lcs_scaled[100][0:5])
print(means[100])
#print(lcs_scaled[1])

[[8.15391918e-03 1.44299832e+00]
 [5.18772603e-05 1.53424018e+00]
 [2.73398658e-03 1.11202903e+00]
 [5.51509397e-03 1.20375259e+00]
 [1.91030567e-02 3.78243564e-01]]
[18.44784829]


In [17]:
main_input = Input(shape=(lcs_scaled.shape[1], 2), name='main_input') #(lag, mag)
aux_input = Input(shape=(lcs_scaled.shape[1], 1), name='aux_input') #(lag)
#print(aux_input.shape)

In [18]:
model_input = [main_input, aux_input]

sample_weight = 1. / errors
#print(sample_weight)
sample_weight[np.isnan(sample_weight)] = 0.0
lcs_scaled[np.isnan(lcs_scaled)] = 0.

#lcs_scaled[:, :, [1]]

In [19]:
#Initialization 
lr = 1e-3 #learning rate 
optimizer = Adam(lr=lr)
#output_size=16
output_size =2
#gru_size = 32
gru_size =10
#nepochs = 2000
nepochs =1000
#batchsize = 512
batchsize =100
#dropout_val = 0.25
dropout_val = 0.20

resume_training = False # if True use W&B to recover weights and resume training, if False train a new model

def sampling(samp_args):
    z_mean, z_log_sigma = samp_args

    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_sigma) * epsilon
    

/home/astro-s6/anaconda3/lib/python3.7/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [20]:
#encoder
encoder = Bidirectional(GRU(gru_size, name='encoder1', return_sequences=True))(main_input)
encoder = Dropout(dropout_val, name='drop_encoder1')(encoder) 
encoder = Bidirectional(GRU(gru_size, name='encoder2', return_sequences=False))(encoder)
encoder = Dropout(dropout_val, name='drop_encoder2')(encoder)
codings_mean = Dense(units=output_size, name='encoding_mean', activation='linear')(encoder)
codings_log_var = Dense(units=output_size, name='encoding_log_var', activation='linear')(encoder) 
codings = Lambda(sampling, output_shape=(output_size,))([codings_mean, codings_log_var])


#decoder
decoder = RepeatVector(lcs_scaled.shape[1], name='repeat')(codings)
#decoder = tf.keras.layers.merge.concatenate([aux_input, decoder])
decoder = tf.keras.layers.concatenate([aux_input, decoder])
decoder = GRU(gru_size, name='decoder1', return_sequences=True)(decoder)
decoder = Dropout(dropout_val, name='drop_decoder1')(decoder)
decoder = GRU(gru_size, name='decoder2', return_sequences=True)(decoder)
decoder = TimeDistributed(Dense(1, activation='linear'), name='time_dist')(decoder)

#VAE
model = Model(model_input, decoder)

latent_loss = -0.5*K.sum(1+codings_log_var-K.exp(codings_log_var)-K.square(codings_mean),axis=-1)
model.add_loss(K.mean(latent_loss)/200.)
model.compile(optimizer=optimizer, loss='mse',  metrics=[tf.keras.metrics.MeanAbsoluteError()], weighted_metrics=[tf.keras.metrics.MeanAbsoluteError()], sample_weight_mode='temporal')#,run_eagerly=True)

model.summary()


2022-08-23 16:07:17.829954: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-23 16:07:17.829974: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-23 16:07:17.829988: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dharmender): /proc/driver/nvidia/version does not exist
2022-08-23 16:07:17.830534: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 main_input (InputLayer)        [(None, 1850, 2)]    0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 1850, 20)     840         ['main_input[0][0]']             
                                                                                                  
 drop_encoder1 (Dropout)        (None, 1850, 20)     0           ['bidirectional[0][0]']          
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 20)          1920        ['drop_encoder1[0][0]']          
 )                                                                                            

In [21]:
log_dir = './lcs_image/'
weights_path = os.path.join(log_dir, 'weights_lr1e3_4paper.h5')
logs = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))


In [22]:
early_stopping = EarlyStopping(monitor='val_loss', patience=50, mode='auto', restore_best_weights=True)
print(early_stopping)
check_points = tf.keras.callbacks.ModelCheckpoint(filepath=weights_path, save_freq='epoch', save_weights_only=False, save_best_only=True, monitor='val_loss', verbose=1)


In [ ]:
#aux input contains the delta times, Y (lcs_scaled[:, :, [1]]) is the arrey with the normalized magnitudes, sample_weights is 1/err
history = model.fit({'main_input': lcs_scaled, 'aux_input': np.delete(lcs_scaled, 1, axis=2)},
                    lcs_scaled[:, :, [1]], 
                    epochs=nepochs, 
                    batch_size=batchsize,
                    sample_weight=sample_weight,
                    callbacks = [
                                check_points,
                                early_stopping
                                 ],
                    validation_split=0.2,
                    )
                    

2022-08-23 16:07:29.664625: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 217634000 exceeds 10% of free system memory.


Epoch 1/1000
148/148 [==============================] - ETA: 0s - loss: 0.4083 - mean_absolute_error: 0.2884 - weighted_mean_absolute_error: 0.7397
Epoch 1: val_loss improved from inf to 0.40822, saving model to ./lcs_image/weights_lr1e3_4paper.h5
148/148 [==============================] - 269s 2s/step - loss: 0.4083 - mean_absolute_error: 0.2884 - weighted_mean_absolute_error: 0.7397 - val_loss: 0.4082 - val_mean_absolute_error: 0.2822 - val_weighted_mean_absolute_error: 0.7438
Epoch 2/1000
 20/148 [===>..........................] - ETA: 3:41 - loss: 0.3948 - mean_absolute_error: 0.2727 - weighted_mean_absolute_error: 0.7439

In [1]:
train_loss = history.history['loss']
print('the train loss is',train_loss)
val_loss   = history.history['val_loss']
print('the validation loss is',val_loss)
train_mae  = history.history['mean_absolute_error']
val_mae    = history.history['val_mean_absolute_error']
train_wmae  = history.history['weighted_mean_absolute_error']
val_wmae    = history.history['val_weighted_mean_absolute_error']
xc         = range(len(train_loss))


NameError: name 'history' is not defined

In [ ]:
df_history = pd.DataFrame({'epoch': xc, 'train_loss': train_loss, 'val_loss': val_loss, 
                           'train_mae': train_mae, 'val_mae': val_mae,
                           'train_wmae': train_wmae, 'val_wmae': val_wmae})

history_path = os.path.join(log_dir, 'train_history_4paper_lr1e3.csv')


df_history.to_csv(history_path)


In [ ]:
#print(xc)
#data=pd.read_csv(history_path)
#print(data)
print(val_loss)

In [ ]:
plt.figure()
plt.plot(xc, train_loss)
plt.plot(xc, val_loss)
plt.savefig('./lcs_image/xc_val_loss_and_train_loss.png')

model_loss = pd.DataFrame(model.history.history)
model_loss.plot()
plt.savefig('./lcs_image/GRU2x32-encoding16_loss.png')


In [ ]:
encode_model = Model(model.input, model.get_layer('lambda').output)
decode_model = Model(model.input, model.output)
print(encode_model)
print(decode_model)
encoding = encode_model.predict({'main_input': lcs_scaled, 'aux_input': np.delete(lcs_scaled, 1, axis=2)})
print(encoding)
decoding = decode_model.predict({'main_input': lcs_scaled, 'aux_input': np.delete(lcs_scaled, 1, axis=2)})
print(decoding)

In [ ]:
encoding_train = encoding[0:int(0.8*len(lcs))]
print(encoding_train)
encoding_val = encoding[int(0.8*len(lcs)):len(lcs)]
decoding_train = decoding[0:int(0.8*len(lcs))]
decoding_val = decoding[int(0.8*len(lcs)):len(lcs)]
X_raw_train = lcs_raw[0:int(0.8*len(lcs))]
X_raw_val = lcs_raw[int(0.8*len(lcs)):len(lcs)]
X_train = lcs_scaled[0:int(0.8*len(lcs))]
X_val = lcs_scaled[int(0.8*len(lcs)):len(lcs)]
scales_train = scales[0:int(0.8*len(lcs))]
scales_val = scales[int(0.8*len(lcs)):len(lcs)]
means_train = means[0:int(0.8*len(lcs))]
means_val = means[int(0.8*len(lcs)):len(lcs)]

In [ ]:
i = 150 #init num of the plotting data
#i=15000
k=0 #initialize the counter
fignum = 5
plt.figure(figsize=(12, 4))
#plotting raw-lightcurves and the decoded lightcurves 
for num in range(i,i+fignum):
    plt.subplot(2, fignum, k+1)
    #print(X_raw_train[num][:,0])
    #print(365.0*np.cumsum(X_train[num][:, 0][X_train[num][:, 0]>0])+X_raw_train[num][0,0])
    plt.errorbar(X_raw_train[num][:,0], X_raw_train[num][:,1], yerr=X_raw_train[num][:, 2], fmt='o', color='black', ms=5)
    plt.errorbar(365.0*np.cumsum(X_train[num][:, 0][X_train[num][:, 0]>0])+X_raw_train[num][0,0], 
             decoding_train[num][X_train[num][:, 0]>0]*scales_train[num]+means_train[num], fmt='o', color='orange', ms=7)
    plt.ylim(plt.ylim()[::-1])
    k += 1
#plotting encoded features
k=0
for num in range(i+fignum,i+2*fignum):
    plt.subplot(2, fignum, k+1+fignum)
    plt.imshow(encoding_train[num].reshape(4,4), vmin=-2, vmax=2, cmap='viridis_r')
    k += 1
plt.tight_layout()
plt.savefig('./lcs_image/ZTF_alerts_modeled_lcs_training_set.png')
plt.show()

print("""Upper panels: Input light curves (black points) and decoded light curves (orange points) \n
Lower panels: Encoded 16 features""")


In [ ]:
#i = 1650 #init num of the plotting data
i=16
k=0 #initialize the counter
fignum = 5
plt.figure(figsize=(12, 4))
#plotting raw-lightcurves and the decoded lightcurves 
for num in range(i,i+fignum):
    plt.subplot(2, fignum, k+1)
    plt.errorbar(X_raw_val[num][:,0], X_raw_val[num][:,1], yerr=X_raw_val[num][:, 2], fmt='o', color='black', ms=5)
    plt.errorbar(365.0*np.cumsum(X_val[num][:, 0][X_val[num][:, 0]>0])+X_raw_val[num][0,0], 
             decoding_val[num][X_val[num][:, 0]>0]*scales_val[num]+means_val[num], fmt='o', color='orange', ms=7)
    plt.ylim(plt.ylim()[::-1])
    k += 1
#plotting encoded features
k=0
for num in range(i+fignum,i+2*fignum):
    plt.subplot(2, fignum, k+1+fignum)
    plt.imshow(encoding_val[num].reshape(4,4), vmin=-2, vmax=2, cmap='viridis_r')
    k += 1
plt.tight_layout()
plt.savefig('./lcs_image/ZTF_alerts_modeled_lcs_validation_set.png')
plt.show()

In [ ]:
def calc_redchisq(x, x_pred, weight):
    mask =  (~np.isnan(weight))
    out = np.sum(((x[mask]-x_pred[mask])*weight[mask])**2)/len(weight[mask])
    return out


In [ ]:
RedChiSq = []
print('Caluculating reduced chi-square for each source')
for m in range(0, len(lcs_raw)):
    if m%100 == 0:
        print('.', end='')
    RedChiSq.append(calc_redchisq(lcs_scaled[m][:,1]*scales[m]+means[m], np.squeeze(decoding[m]*scales[m]+means[m]), 1/lcs_raw[m][:,2]))
    
RedChiSq_train = RedChiSq[0:int(0.8*len(lcs))]
RedChiSq_val = RedChiSq[int(0.8*len(lcs)):len(lcs)]    

print("training chi2 median, mean: ",np.median(np.array(RedChiSq_train)),np.mean(np.array(RedChiSq_train)))
print("validation chi2 median, mean: ",np.median(np.array(RedChiSq_val)),np.mean(np.array(RedChiSq_val)))


In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.hist(np.array(RedChiSq_train)[~np.isnan(RedChiSq_train)], 
         bins=10**np.arange(-1, 3, 0.1))

In [ ]:
plt.xscale('log')
plt.xlabel(r'$\chi_{{\rm red}}^2$', fontsize=15)
plt.ylabel('Number of sources', fontsize=15)
plt.title('Training dataset', fontsize=15)
plt.tight_layout()


In [ ]:
plt.subplot(1,2,2)
plt.hist(np.array(RedChiSq_val)[~np.isnan(RedChiSq_val)], 
         bins=10**np.arange(-1, 3, 0.1))

In [ ]:
plt.xscale('log')
plt.xlabel(r'$\chi_{{\rm red}}^2$', fontsize=15)
plt.ylabel('Number of sources', fontsize=15)
plt.title('Validation dataset', fontsize=15)
plt.tight_layout()

In [ ]:
plt.savefig('./lcs_image/ZTF_alerts_chisquare_distribution.png')
plt.show()

In [ ]:
X_reduced_train =TSNE(n_components=2, perplexity=100, random_state=32, n_iter=1000).fit_transform(encoding_train)